# 1. Population Genetics Simulation

Create a program that simulates the allelic frequency in a finite diploid population for a certain number of generations.

The program takes as input the initial allele frequencies, the fitness of each genotype, the population size, and the number of generations. Because these simulations are stochastic each run of the simulation will give a different result, to allow an idea of the behavior of the allelic frequencies, your program should repeat the simulations many times for each parameter set and plot all the results in a single graph. The number of simulations should also be determined by the user. You can start your program using the variable definitions in the cell below.

Your program should output two graphs. The first should show the allele frequency at each generation, and the other should be a histogram with the final values of the allele frequency. Something like this:

![simulation](Sim1.png)

![histogram](Sim2.png)

Last year a student used this homework as the starting point for her project to create a population genetics simulator for BIOL040. You can see the final project here: http://dna.pomona.edu:5006/pop_gen_sim

In [3]:
import numpy as np
import plotly.graph_objects as go

# Allele frequencies
initA = 0.50
inita = 0.50  # 1 - inita

# Fitnesses
fAA = 1
fAa = 1
faa = 1

# Population Size
pop = 1000

# Number of Generations
gen = 100

# Number of Simulations
sim = 100


#plot starts from y=0.5 where y=f_A

            #AA             Aa            aa
#f_A      f_A^2          2f_Af_a        fa^2   --> 1
#f_a     W_AAfA^2      W_aa2f_af_a     Waafa^2   --> <1
   # => W_AAf_A^2/M   W_aaf_af_a/M   Waafa^2/M
     #    f(t+1)_AA     f(t+1)_Aa     f(t+1)_aa    expected frequency of each genotype
#random binomial distribution
#for Gen t+1   f(t+1)_A = f(t+1)_AA + 1/2f(t+1)_Aa
            #  f(t+1)_a = 1 - f(t+1)_A
#dirichlet distribution 
#s = np.random.binomial(population size, frequency of genotype in interest, 1)

def simulate_population(init_A, init_a, f_AA, f_Aa, f_aa, population_size, generations):
    allele_freq_results = []
    
    for _ in range(sim):
        allele_freq = [init_A]
        
        #simulate for a number of generations
        for _ in range(generations):

            #set up the allele frequencies
            f_A = allele_freq[-1]
            f_a = 1 - f_A

            # Calculate expected genotype frequencies
            expected_f_AA = f_AA * f_A**2
            expected_f_Aa = 2 * f_A * f_a * f_Aa
            expected_f_aa = f_aa * f_a**2

            # Simulate genotype frequencies with binomial distribution
            f_AA_new = np.random.binomial(population_size, expected_f_AA) 
            f_Aa_new = np.random.binomial(population_size, expected_f_Aa) 
            f_aa_new = np.random.binomial(population_size, expected_f_aa) 

            # Normalize the genotype frequencies
            total_freq = f_AA_new + f_Aa_new + f_aa_new
            f_AA_new /= total_freq
            f_Aa_new /= total_freq
            f_aa_new /= total_freq

            # Update allele frequency for the next generation
            f_A_new = f_AA_new + 0.5 * f_Aa_new

            #add the new frequncy to the frequency list
            allele_freq.append(f_A_new)

        #add to the new result
        allele_freq_results.append(allele_freq)

    return allele_freq_results

# Run simulations
allele_freq_results = simulate_population(initA, inita, fAA, fAa, faa, pop, gen)

# Create the first graph showing allele frequency at each generation
fig1 = go.Figure()

for i, allele_freq in enumerate(allele_freq_results):
    fig1.add_trace(go.Scatter(x=list(range(gen + 1)), y=allele_freq, mode='lines', name=f'Simulation {i + 1}'))

fig1.update_layout(
    title='Allele Frequency Simulation',
    xaxis_title='Generations',
    yaxis_title='Allele Frequency',
    showlegend=True
)

fig1.show()

# Create the second histogram graph with the final values of the allele frequency
final_allele_freq = [allele_freq[-1] for allele_freq in allele_freq_results]

fig2 = go.Figure()

fig2.add_trace(go.Histogram(x=final_allele_freq,nbinsx=20))

fig2.update_layout(
    title='Histogram of Final Allele Frequencies',
    xaxis_title='Final Allele Frequency',
    yaxis_title='Frequency',
    showlegend=False
)

fig2.update_xaxes(range=[0, 1])  # Set x-axis range from 0 to 1

fig2.show()
